# Probe the internal representation of diffusion model

Before running this notebook, please run [create_the_synthetic_dataset.ipynb](create_the_synthetic_dataset.ipynb) to first create the dataset for probing.

Also make sure you have switch to the right conda environment (kernel). If you create the conda environment using the provided environment.yml file, the name of that environment should be `diffusion-viz-gpu`. Switch the notebook kernel to `diffusion-viz-gpu` before running the cells below.

In addition to running the bash commands below in the jupyter notebook, you may paste them into a terminal for execution.

## 1. Probe the Representation of Saliency (Foreground)

Once you create the dataset, you could run the code below to probe the representation of salient regions (foreground) in the LDM

In [ ]:
%%sh
for step in {0..14}
do
    python probing_binary_depth.py --layer_name "transformer_blocks.0.attn1.to_out.0" --step $step --block_type "attentions" --output_dir "attn1_out" --postfix "self_attn_out"
done

## 2. Probe the Representation of Depth

Run the code below to probe the LDM's representation of inverse depth.

`--probe_type "Linear-no-bias"` means we are using a linear regressor without the bias term when probing the depth representation

In [ ]:
%%sh

for step in {0..14}
do
    python probing_continuous_depth.py --layer_name "transformer_blocks.0.attn1.to_out.0" --step $step --block_type "attentions" --output_dir "attn1_out" --postfix "self_attn_out" --probe_type "Linear-no-bias"
done

## 3. Controlled Experiment

The code below train the linear classifiers and regressors on a randomized latent diffusion model. The probing performance on the randomized model served as the baseline.

In [ ]:
%%sh

# Baseline experiment for salient object segmentation task (saliency representation)
for step in {0..14}
do
    python probing_binary_depth_controlled.py --layer_name "transformer_blocks.0.attn1.to_out.0" --step $step --block_type "attentions" --output_dir "attn1_out" --postfix "self_attn_out"
done

# Baseline experiment for monocular depth estimation task (depth representation)
for step in {0..14}
do
    python probing_continuous_depth_controlled.py --layer_name "transformer_blocks.0.attn1.to_out.0" --step $step --block_type "attentions" --output_dir "attn1_out" --postfix "self_attn_out" --probe_type "Linear-no-bias"
done

## 4. Probe Representation of Saliency and Depth in VAE

VAE also has one self-attention layer in the bottleneck of its Decoder. The code below explore the saliency and depth representations in VAE's self-attention layer.

In [ ]:
# Probe the representation of salient regions in VAE self-attention
for step in {0..14}
do
    python probing_binary_depth_vae.py --layer_name "proj_attn" --step $step --block_type "attentions" --output_dir "attn1_out" --postfix "self_attn_out"
done

# Probe the representation of depth in VAE self-attention
for step in {0..14}
do
    python probing_continuous_depth_vae.py --layer_name "proj_attn" --step $step --block_type "attentions" --output_dir "attn1_out" --postfix "self_attn_out" --probe_type "Linear-no-bias"
done